In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load Dataset
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Data Overview
print(train_df.head())
print(train_df.info())
print(train_df.describe())

# Fill missing values
train_df['Age'].fillna(train_df['Age'].median(), inplace=True)
train_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace=True)
test_df['Age'].fillna(test_df['Age'].median(), inplace=True)
test_df['Fare'].fillna(test_df['Fare'].median(), inplace=True)

# Encode categorical variables
label_encoder = LabelEncoder()
train_df['Sex'] = label_encoder.fit_transform(train_df['Sex'])
train_df['Embarked'] = label_encoder.fit_transform(train_df['Embarked'])
test_df['Sex'] = label_encoder.transform(test_df['Sex'])
test_df['Embarked'] = label_encoder.transform(test_df['Embarked'])

# Normalize numerical features
scaler = StandardScaler()
train_df[['Age', 'Fare']] = scaler.fit_transform(train_df[['Age', 'Fare']])
test_df[['Age', 'Fare']] = scaler.transform(test_df[['Age', 'Fare']])

# Feature Engineering
train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch'] + 1
test_df['FamilySize'] = test_df['SibSp'] + test_df['Parch'] + 1

# Data Visualization
sns.countplot(x='Survived', data=train_df)
plt.show()

sns.countplot(x='Pclass', hue='Survived', data=train_df)
plt.show()

sns.countplot(x='Sex', hue='Survived', data=train_df)
plt.show()

sns.histplot(train_df['Age'], bins=30, kde=True)
plt.show()

# Model Building
X = train_df.drop(['Survived', 'Name', 'Ticket', 'Cabin'], axis=1)
y = train_df['Survived']

X_train, X_val, y_train, y_val = train_test_split(X, y)